In [1]:
import numpy as np
import torch


In [2]:
def get_parallel_db(tensor, remove_index):
    "remove an entry at the associated index"
    return torch.cat( (tensor[0:remove_index], tensor[remove_index+1:]) )

def create_dbs_parallel(database):
    "remove an entry (user) from muliple different variations of queries "
    pdbs = [get_parallel_db(database, i) for i in range(len(database)) ]
    return pdbs

num_entries = 10
db   = torch.rand(num_entries) > 0.5
db   = db.type(torch.IntTensor)
pdbs = create_dbs_parallel(db)

# Sensitivity
- determine sensitivity of information that changes; how queries leak information 
- L1 Sensitivity: maximum amount query changes when removing an individual from the database

In [3]:
# query database and measure the privacy of the query to determine if result is leaking information 
# does the output change when removing an (entry=user) from the database
def query_statistic(database, fn_metric):
    return fn_metric(database)

In [7]:
apriori_min, apriori_max = (min(db), max(db))
sensitivity = 0
db_full_stat = query_statistic(db, torch.sum)
for pdb in pdbs:
    pdb_result = query_statistic(pdb, torch.sum)
    pdb_dist   = torch.abs(pdb_result - db_full_stat)
    if pdb_dist > sensitivity:
        sensitivity = pdb_dist
sensitivity    

tensor(1)

In [8]:
apriori_min, apriori_max = (min(db), max(db))
db_full_stat   = query_statistic(db, torch.sum)
sensity_fn_l1  = np.max
sensitvity_val = sensity_fn_l1([torch.abs(query_statistic(pdb, torch.sum) - db_full_stat) for pdb in pdbs])
sensitvity_val == apriori_max, sensitvity_val

(tensor(True), 1)